In [2]:
import tensorflow as tf
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

def buildLayer(_inputTensor, _numUnits):
    '''
    input:
    _inputTensor S = θ(Xprev)from the prev layer
    _numUnits is the num of neurons in this layer
    
    intermediate xavierInit:
        W is initialized as Xavier
        W is input.shape[1] by num units
    
    output:
    weighted sum of inputs
    
    '''
    
    # zero mean independent Gaussians whose variance is 3/(#input + #outputs)
    dim1 = _inputTensor.get_shape()[0]
    W = tf.get_variable("a", shape = [dim1, _numUnits], initializer=tf.contrib.layers.xavier_initializer())
    b = tf.Variable(0.0, name='biases')
    
    
if __name__ == '__main__':  
    
    a = tf.constant([3.0, 2.0, 4.0, 5.0])
    b = 3
    xavierInit(a, b)
    

4
<tf.Variable 'a:0' shape=(4, 3) dtype=float32_ref>
